In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_bvp

## Set Up

In [ ]:
"""
CONSTANTS
mass in kg
distance in km
force in kN
angle in degrees
speed in km/h
time in hours
"""

# ROCKET
Mr0 = 5.49054e5         # initial mass of rocket
rocket_height = 70      # height of the rocket (not distance from earth)
rocket_diameter = 3.7   # max diameter of rocket
thrust_max1 = 7.56e6    # max thrust of stage 1
thrust_max2 = 9.81e2    # max thrust of stage 2
c = 0.25                # coefficient of drag for the rocket
A = rocket_diameter * np.pi # area that the wind hits

# EARTH
Ve = 0                  # velocity of the earth
Me = 5.97219e24         # mass of the earth
we = 1.675e3            # angular velocity of the earth at the equator
Re = 6.3781e3           # radius of the earth

# MOON
Vm = 3.683e3            # speed of the moon
Mm = 7.34767309e22      # mass of the moon
wm = 1.67e1             # angular velocity of the moon at its equator
Rm = 1.738e3            # radius of the moon
P = 6.552e2             # period of the orbit of the moon

# SPACE
Lm = 3.84e5             # distance from the earth to the moon
G = 6.673e-11           # graviational constant in (N * m^2) / (k * g^2)

# DRAG
p0 = 1.01325e5          # atmospheric pressure at sea level in Pascals
Hn = 1.04e1             # constant for drag force 
K = 1e2                 # Karman line, when drag is zero and space begins

ASSUMPTIONS
1. Rocket, earth, and moon have point mass for the sake of calculating gravity
2. Rocket, earth, and moon are the only objects for the sake of calculating gravity (no influence of the sun, stars, etc)
3. Rocket, earth, and moon lie on a 2D plane
4. Moon's orbit is perfectly circular
5. Rocket's mass does not change
6. Rocket is single stage

ADDITIONAL COMPLEXITY
1. Rocket's mass changes over time due to propellent use
2. Moon's orbit is slightly elliptical
3. Find orbital path in 3D space
4. Calculate optimal time to take off
5. Land on the moon
6. Take off from somewhere besides the equate and orbit the moon someone besides the equator
7. Calculate optimal time to release second stage

REDUCED COMPLEXITY
1. Moon does not move

In [ ]:
# initial variables and endpoints
# position
xrx0 = Re               # initial x position of rocket
xry0 = 0                # initial y position of rocket
xex0 = 0                # initial x position of earth
xey0 = 0                # initial y position of earth
xmx0 = Lm               # initial x position of moon
xmy0 = 0                # initial y position of moon
# velocity
vrx0 = 0                # initial x velocity of rocket
vry0 = we               # initial y velocity of rocket
vex0 = 0                # initial x velocity of earth
vey0 = 0                # initial y velocity of earth
vmx0 = 0                # initial x velocity of moon
vmy0 = Vm               # initial y velocity of moon
# initial acceleration is all zero

# position
xr0 = np.array([xrx0, xry0])     # IC
xe0 = np.array([xex0, xey0])
xm0 = np.array([xmx0, xmy0])
# velocity
vr0 = np.array([vrx0, vry0])     # IC
ve0 = np.array([vex0, vey0])
vm0 = np.array([vmx0, vmy0])
# acceleration
ar0 = np.array([0,0])
ae0 = np.array([0,0])
am0 = np.array([0,0])

tht0 = 0                # initial angle between earth and rocket
phi0 = 0                # initial angle between earth and moon

# control
ux0 = 0                 # initial x thrust of rocket
uy0 = 0                 # initial y thrust of rocket
u = np.array([ux0, uy0])


# final endpoints. idk bro
hf = 1e1                # final distance between rocket and moon
vf = 1e1                # orbital speed of the moon but idk what that is
# xrxf = moon_position(tf) * np.cos(phi_tf) + hf * np.cos(phi_tf + 90)
# xryf = moon_position(tf) * np.sin(phi_tf) + hf * np.sin(phi_tf + 90)
# vrxf = Vm * np.cos(phi_tf) + vf * np.cos(phi_tf + 90)
# vryf = Vm * np.sin(phi_tf) + vf * np.sin(phi_tf + 90)

# xrf = np.array([xrxf, xryf])
# vrf = np.array([vrxf, vryf])

## Update state equations

### Helper Helper functions

In [ ]:
def find_angle(pos):
    return np.arcsin(pos[1] / np.linalg.norm(pos))

def er_cos(d, theta, phi):
    """
    Calculate the cosine of the angle between the rocket, earth, and the vertical.

    @param d: (float) Distance from the rocket to the center of the Earth.
    @param theta: (float) Angle between the rocket's position vector and the horizontal axis.
    @param phi: (float) Angle between the moon's position vector and the center of the Earth.

    @return: (float) The cosine of the angle.
    """
    # note: cos(i)
    return (d**2 + (d*np.sin(theta) - d*np.cos(theta)*np.tan(phi) )**2 - (d*np.cos(theta) / np.cos(phi))**2)


def em_sin(pos, moon, theta, phi):
    """
    Calculate the sine of the angle between the rocket, Earth, and the Moon.

    @param pos: (ndarray) Position vector of the rocket.
    @param moon: (ndarray) Position vector of the Moon.
    @param theta: (float) Angle between the rocket's position vector and the horizontal axis.
    @param phi: (float) Angle between the moon's position vector and the center of the Earth.

    @return: (float) The sine of the angle.
    """
    # note: sin(x)
    return Lm / np.linalg.norm(moon - pos) * np.sin(theta - phi)


def er_sin(d, theta, phi):
    """
    Calculate the sine of the angle between the rocket, Earth, and the vertical.

    @param d: (float) Distance from the rocket to the center of the Earth.
    @param theta: (float) Angle between the rocket's position vector and the horizontal axis.
    @param phi: (float) Angle between the Moon and the center of the Earth.

    @return: (float) The sine of the angle.
    """
    # note: sin(i)
    return d * np.cos(theta) * np.sin(theta) / (d*np.sin(theta) - d*np.cos(theta)*np.tan(phi))


def em_cos(d, pos, moon):
    """
    Calculate the cosine of the angle between the rocket, Earth, and the Moon.

    @param d: (float) Distance from the rocket to the center of the Earth.
    @param pos: (ndarray) Position vector of the rocket.
    @param moon: (ndarray) Position vector of the Moon.

    @return: (float) The cosine of the angle.
    """
    # note: cos(x)
    dm = np.linalg.norm(moon - pos)
    return (d**2 + dm**2 - Lm**2) / (2*d*dm)


def moon_position(t):
    """
    Calculate the position vector of the Moon at a given time.

    @param t: (float) Time elapsed since the start of the simulation.

    @return: (ndarray) Position vector of the Moon.
    """
    #TODO make it so that the moon starts at an arbitrary position instead of (Lm, 0)
    t_norm = t * (2 * np.pi) / P
    return np.array([Lm*np.cos(t_norm), Lm*np.sin(t_norm)])

### Force helpers

In [ ]:
def rocket_mass(t):
    """
    Calculate the mass of the rocket at a given time.

    @param t: (float) Time elapsed since the start of the simulation.

    @return: (float) Mass of the rocket.
    """
    # TODO this should depend on how much fuel has been spent but just returns a constant for now
    return Mr0


def moon_cos(pos, moon, theta, phi):
    """
    Calculate the cosine component of the force exerted on the rocket by the Moon.

    @param pos: (ndarray) Position vector of the rocket.
    @param moon: (ndarray) Position vector of the Moon.
    @param theta: (float) Angle between the rocket's position vector and the horizontal axis.
    @param phi: (float) Angle between the Moon and the horizontal axis.

    @return: (float) Cosine component of the force.
    """
    d = np.linalg.norm(pos)
    return er_cos(d, theta, phi) * em_sin(pos, moon, theta, phi) - er_sin(d, theta, phi) * em_cos(d, moon)


def moon_sin(pos, moon, theta, phi):
    """
    Calculate the sine component of the force exerted on the rocket by the Moon.

    @param pos: (ndarray) Position vector of the rocket.
    @param moon: (ndarray) Position vector of the Moon.
    @param theta: (float) Angle between the rocket's position vector and the horizontal axis.
    @param phi: (float) Angle between the Moon and the center of the Earth.

    @return: (float) Sine component of the force.
    """
    d = np.linalg.norm(pos)
    return er_cos(d, theta, phi) * em_cos(d, moon) - er_sin(d, theta, phi) * em_sin(pos, moon, theta, phi)


def gravity_mag(x, t, moon=np.array([0, 0])):
    """
    Calculate the magnitude of the gravitational force acting on the rocket.

    @param x: (ndarray) Position vector of the rocket.
    @param t: (float) Time elapsed since the start of the simulation.
    @param moon: (ndarray) Position vector of the Moon. Defaults to 0, indicating Earth's gravity.

    @return: (float) Magnitude of the gravitational force.
    """
    mass = Me if moon==0 else Mm
    return G * mass * rocket_mass(t) * np.linalg.norm(moon - x)


def density(height):
    """
    Calculate the air density at a given altitude.

    @param height: (float) Altitude above sea level.

    @return: (float) Air density at the given altitude.
    """
    return p0 * np.exp(-height / Hn)

### Force functions

In [ ]:
def earth_gravity(x, t, theta, x_dir=True):
    """
    Calculate the gravitational force exerted on the rocket by the Earth.

    @param x: (ndarray) Position vector of the rocket.
    @param t: (float) Time elapsed since the start of the simulation.
    @param theta: (float) Angle between the rocket's position vector and the horizontal axis.
    @param x_dir: (bool) Flag indicating whether to calculate force in the x-direction (True) or y-direction (False). Default is True.

    @return: (float) Magnitude of the gravitational force in the specified direction.
    """
    angle = np.cos(theta) if x_dir else np.sin(theta)
    return gravity_mag(x, t) * angle


def moon_gravity(x, t, moon, theta, phi, x_dir=True):
    """
    Calculate the gravitational force exerted on the rocket by the Moon.

    @param x: (ndarray) Position vector of the rocket.
    @param t: (float) Time elapsed since the start of the simulation.
    @param moon: (ndarray) Position vector of the Moon.
    @param theta: (float) Angle between the rocket's position vector and the horizontal axis.
    @param phi: (float) Angle between the Moon and the center of the Earth.
    @param x_dir: (bool) Flag indicating whether to calculate force in the x-direction (True) or y-direction (False). Default is True.

    @return: (float) Magnitude of the gravitational force in the specified direction.
    """
    direction = moon_cos(x, moon, theta, phi) if x_dir else moon_sin(x, moon, theta, phi)
    return gravity_mag(x, t, moon) * direction


def drag(pos, vel, theta, x=True):
    """
    Compute the force due to drag from the atmosphere. 
    Returns zero if the rocket has passed the Karman line.

    @param: pos (ndarray) - 2D array of floats x position and y position of the rocket in m/s
    @param: vel (ndarray) - x velocity and y velocity of the rocket in m/s
    @param: theta (float) - angle between the y-axis and the rocket in degrees
    @param: x (boolean) - True if the force should be returned in the x direction, false for the y direction

    @returns: force (float) - Force due to drag in the desired direction
    """
    height = np.linalg.norm(pos - Re)
    if height < K:
        i = 0 if x else 1
        angle = np.cos(theta) if x else np.sin(theta)
        return 0.5 * c * A * density(height) * vel[i]**2 * angle**2
    return 0


def ux():
    """
    Control?
    """
    return 0


def uy():
    """
    Control?
    """
    return 0


def acceleration(t, x, dx, moon, x_dir=True):
    """
    Calculate the total acceleration experienced by the rocket.

    @param t: (float) Time elapsed since the start of the simulation.
    @param x: (ndarray) Position vector of the rocket.
    @param dx: (ndarray) Velocity vector of the rocket.
    @param moon: (ndarray) Position vector of the Moon.
    @param x_dir: (bool) Flag indicating whether to calculate acceleration in the x-direction (True) or y-direction (False). Default is True.

    @return: (float) Total acceleration of the rocket in the specified direction.
    """
    control = ux() if x_dir else uy()
    theta = find_angle(x)
    phi = find_angle(moon)
    return control - \
            earth_gravity(x, t, theta, x_dir) + \
            moon_gravity(x, t, moon, theta, phi, x_dir) - \
            drag(x, dx, theta, x_dir)

## ODE

In [ ]:
# Algorithm 21.2 this is copied from my homework as of 3/29/24 lol

# set up the right hand side of the ODE, with the parameter
# tf (the final time rescaled)
r = 1
delta = 0.5
n = 100
N0 = 2
m = -1
f = 1


def chemo_finite_time(t,y,p):
    tf = p[0]
    N = y[0]
    p = y[1]
    u = -p*delta*N / 2
    return np.vstack((tf*(r*N*np.log(1/N) - delta*u*N), 
                      tf*(p*r - p*r*np.log(1/N) + p*delta*u + 2*N)))
# set up the endpoint conditions. The final one is H(tf) = 2a( -tf-1)
def bc_new(ya,yb,p):
    tf = p[0]
    return np.array([ya[0]-N0, yb[1]-m, tf-f])

# construct the solution using solve_bvp. We use an initial guess
# of tf=1 as that is the value we are forcing the solution to be close to.
t = np.linspace(0, 1, n)
y = np.ones((2, t.size))*2
p0 = np.array([2])
res = solve_bvp(chemo_finite_time, bc_new, t, y, p=p0)

# plot the results and print out the optimal final time t_f
t_plot = np.linspace(0, 1, n)
x_plot = res.sol(t_plot)[0]
p_plot = res.sol(t_plot)[1]
u_plot = -p_plot * delta * x_plot / 2
plt.plot(t_plot, x_plot)
plt.plot(t_plot, u_plot)
plt.legend(['cancer concentration','drug dosage'])
print('t_f = '+str(res.p[0]))